### Download data

In [1]:
#!wget --no-check-certificate https://s3-us-west-1.amazonaws.com/tigergraph-benchmark-dataset/LDBC/SF-1/ldbc_snb_data-sf1.tar.gz 

--2020-06-15 14:20:55--  https://s3-us-west-1.amazonaws.com/tigergraph-benchmark-dataset/LDBC/SF-1/ldbc_snb_data-sf1.tar.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.33
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.33|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 283221929 (270M) [application/x-tar]
Saving to: ‘ldbc_snb_data-sf1.tar.gz.1’

ldbc_snb_data-sf1.t 100%[===================>] 270.10M  14.2MB/s    in 23s     

2020-06-15 14:21:19 (11.7 MB/s) - ‘ldbc_snb_data-sf1.tar.gz.1’ saved [283221929/283221929]



In [2]:
#!tar -xzf  ldbc_snb_data-sf1.tar.gz

In [3]:
!ls -l ldbc_snb_data/social_network/

total 1128356
-rw-r--r-- 1 tigergraph tigergraph 211536570 Feb  5  2019 comment_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  54900516 Feb  5  2019 comment_hasCreator_person_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  50398781 Feb  5  2019 comment_hasTag_tag_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  33715224 Feb  5  2019 comment_isLocatedIn_place_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  28431004 Feb  5  2019 comment_replyOf_comment_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  27631951 Feb  5  2019 comment_replyOf_post_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph   6088868 Feb  5  2019 forum_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  27228450 Feb  5  2019 forum_containerOf_post_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph  90964927 Feb  5  2019 forum_hasMember_person_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph   2448045 Feb  5  2019 forum_hasModerator_person_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph   5597697 Feb  5  2019 forum_hasTag_tag_0_0.csv
-rw-r--r-- 1 tigergraph tigergraph    819

In [4]:
import pandas as pd

df = pd.read_csv('ldbc_snb_data/social_network/person_0_0.csv', sep='|', nrows=5)
df

,id,firstName,lastName,gender,birthday,creationDate,locationIP,browserUsed,language,email
0,933,Mahinda,Perera,male,1989-12-03,2010-02-14T15:32:10.447+0000,119.235.7.103,Firefox,si;en,Mahinda933@boarderzone.com;Mahinda933@hotmail....
1,1129,Carmen,Lepland,female,1984-02-18,2010-01-28T06:39:58.781+0000,195.20.151.175,Internet Explorer,et;en,Carmen1129@gmail.com;Carmen1129@yahoo.com
2,4194,Hồ Chí,Do,male,1988-10-14,2010-02-15T00:46:17.657+0000,103.2.223.188,Internet Explorer,vi;en,Ho.Chi4194@gmx.com
3,8333,Rahul,Kumar,female,1980-02-02,2010-02-12T12:21:43.365+0000,27.97.186.123,Internet Explorer,mr;hi;en,Rahul8333@gmx.com;Rahul8333@yahoo.com
4,8698,Rahul,Reddy,female,1982-05-29,2010-01-21T10:44:41.479+0000,27.97.237.23,Firefox,as;te;en,Rahul8698@gmail.com;Rahul8698@gmx.com;Rahul869...


### Define schema

In [5]:
%%writefile schema.gsql

//clear the current catalog. 
// It may take a while since it restarts the subsystem services. 
DROP ALL

//vertex types
CREATE VERTEX Comment (PRIMARY_ID id UINT, id UINT, creationDate DATETIME, locationIP STRING, browserUsed STRING, content STRING, length UINT)
CREATE VERTEX Post (PRIMARY_ID id UINT, id UINT, imageFile STRING, creationDate DATETIME, locationIP STRING, browserUsed STRING, lang STRING, content STRING, length UINT) 
CREATE VERTEX Company (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX University (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX City (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX Country (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX Continent (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX Forum (PRIMARY_ID id UINT, id UINT, title STRING, creationDate DATETIME)
CREATE VERTEX Person (PRIMARY_ID id UINT, id UINT, firstName STRING, lastName STRING, gender STRING, birthday DATETIME, creationDate DATETIME, locationIP STRING, browserUsed STRING, speaks set<STRING>, email set<STRING>)
CREATE VERTEX Tag (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)
CREATE VERTEX TagClass (PRIMARY_ID id UINT, id UINT, name STRING, url STRING)

//edge types
CREATE DIRECTED EDGE Forum_CONTAINER_OF_Post (FROM Forum, TO Post) WITH REVERSE_EDGE="Forum_CONTAINER_OF_Post_REVERSE"
CREATE DIRECTED EDGE Comment_HAS_CREATOR_Person (FROM Comment, TO Person) WITH REVERSE_EDGE="Comment_HAS_CREATOR_Person_REVERSE"
CREATE DIRECTED EDGE Post_HAS_CREATOR_Person (FROM Post, TO Person) WITH REVERSE_EDGE="Post_HAS_CREATOR_Person_REVERSE"
CREATE DIRECTED EDGE Person_HAS_INTEREST_Tag (FROM Person, TO Tag) WITH REVERSE_EDGE="Person_HAS_INTEREST_Tag_REVERSE"
CREATE DIRECTED EDGE Forum_HAS_MEMBER_Person (FROM Forum, TO Person, joinDate DATETIME) WITH REVERSE_EDGE="Forum_HAS_MEMBER_Person_REVERSE"
CREATE DIRECTED EDGE Forum_HAS_MODERATOR_Person (FROM Forum, TO Person) WITH REVERSE_EDGE="Forum_HAS_MODERATOR_Person_REVERSE"
CREATE DIRECTED EDGE Comment_HAS_TAG_Tag (FROM Comment, TO Tag) WITH REVERSE_EDGE="Comment_HAS_TAG_Tag_REVERSE"
CREATE DIRECTED EDGE Post_HAS_TAG_Tag (FROM Post, TO Tag) WITH REVERSE_EDGE="Post_HAS_TAG_Tag_REVERSE"
CREATE DIRECTED EDGE Forum_HAS_TAG_Tag (FROM Forum, TO Tag) WITH REVERSE_EDGE="Forum_HAS_TAG_Tag_REVERSE"
CREATE DIRECTED EDGE Tag_HAS_TYPE_TagClass (FROM Tag, TO TagClass) WITH REVERSE_EDGE="Tag_HAS_TYPE_TagClass_REVERSE"
CREATE DIRECTED EDGE Company_IS_LOCATED_IN_Country (FROM Company, TO Country) WITH REVERSE_EDGE="Company_IS_LOCATED_IN_Country_REVERSE"
CREATE DIRECTED EDGE Comment_IS_LOCATED_IN_Country (FROM Comment, TO Country) WITH REVERSE_EDGE="Comment_IS_LOCATED_IN_Country_REVERSE"
CREATE DIRECTED EDGE Post_IS_LOCATED_IN_Country (FROM Post, TO Country) WITH REVERSE_EDGE="Post_IS_LOCATED_IN_Country_REVERSE"
CREATE DIRECTED EDGE Person_IS_LOCATED_IN_City (FROM Person, TO City) WITH REVERSE_EDGE="Person_IS_LOCATED_IN_City_REVERSE"
CREATE DIRECTED EDGE University_IS_LOCATED_IN_City (FROM University, TO City) WITH REVERSE_EDGE="University_IS_LOCATED_IN_City_REVERSE"
CREATE DIRECTED EDGE City_IS_PART_OF_Country (FROM City, TO Country) WITH REVERSE_EDGE="City_IS_PART_OF_Country_REVERSE"
CREATE DIRECTED EDGE Country_IS_PART_OF_Continent (FROM Country, TO Continent) WITH REVERSE_EDGE="Country_IS_PART_OF_Continent_REVERSE"
CREATE DIRECTED EDGE TagClass_IS_SUBCLASS_OF_TagClass (FROM TagClass, TO TagClass) WITH REVERSE_EDGE="TagClass_IS_SUBCLASS_OF_TagClass_REVERSE"
CREATE DIRECTED EDGE Person_KNOWS_Person (FROM Person, TO Person, creationDate DATETIME) WITH REVERSE_EDGE="Person_KNOWS_Person_REVERSE"
CREATE DIRECTED EDGE Person_LIKES_Comment (FROM Person, TO Comment, creationDate DATETIME) WITH REVERSE_EDGE="Person_LIKES_Comment_REVERSE"
CREATE DIRECTED EDGE Person_LIKES_Post (FROM Person, TO Post, creationDate DATETIME) WITH REVERSE_EDGE="Person_LIKES_Post_REVERSE"
CREATE DIRECTED EDGE Comment_REPLY_OF_Comment (FROM Comment, TO Comment) WITH REVERSE_EDGE="Comment_REPLY_OF_Comment_REVERSE"
CREATE DIRECTED EDGE Comment_REPLY_OF_Post (FROM Comment, TO Post) WITH REVERSE_EDGE="Comment_REPLY_OF_Post_REVERSE"
CREATE DIRECTED EDGE Person_STUDY_AT_University (FROM Person, TO University, classYear INT) WITH REVERSE_EDGE="Person_STUDY_AT_University_REVERSE"
CREATE DIRECTED EDGE Person_WORK_AT_Company (FROM Person, TO Company, workFrom INT) WITH REVERSE_EDGE="Person_WORK_AT_Company_REVERSE"

//LDBC SNB graph schema 
CREATE GRAPH ldbc_snb (*)

Overwriting schema.gsql


In [6]:
!gsql schema.gsql

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph test, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.

Shutdown restpp gse gpe ...
Graph store /home/tigergraph/tigergraph/gstore/0/ has been cleared!
Everything is dropped.
The vertex type Comment is created.
The vertex type Post is created.
The vertex type Company is created.
The vertex type University is created.
The vertex type City is created.
The vertex type Country is created.
The vertex type Continent is created.
The vertex type Forum is created.
The vertex type Person is created.
The vertex type Tag is created.
The vertex type TagClass is created.
The edge type Forum_CONTAINER_OF_Post is created.
The reverse edge type Forum_CONTAINER_OF_Post_REVERSE is created.
The edge type Comment_HAS_CREATOR_Person is created.
The reverse edge type Comment_HAS_CREATOR_Person_REVERSE is created.
The edge type Post_HAS_CREATOR_Person is created.
The reverse edge type P

### Create load job and run it

In [7]:
%%writefile create_load_job.gsql

USE GRAPH ldbc_snb
CREATE LOADING JOB load_ldbc_snb FOR GRAPH ldbc_snb {
  // define vertex source files
  DEFINE FILENAME v_comment_file;
  DEFINE FILENAME v_post_file;
  DEFINE FILENAME v_organisation_file;
  DEFINE FILENAME v_place_file;
  DEFINE FILENAME v_forum_file;
  DEFINE FILENAME v_person_file;
  DEFINE FILENAME v_tag_file;
  DEFINE FILENAME v_tagclass_file;
  
  // define edge source files
  DEFINE FILENAME forum_containerOf_post_file;
  DEFINE FILENAME comment_hasCreator_person_file;
  DEFINE FILENAME post_hasCreator_person_file;
  DEFINE FILENAME person_hasInterest_tag_file;
  DEFINE FILENAME forum_hasMember_person_file;
  DEFINE FILENAME forum_hasModerator_person_file;
  DEFINE FILENAME comment_hasTag_tag_file;
  DEFINE FILENAME post_hasTag_tag_file;
  DEFINE FILENAME forum_hasTag_tag_file;
  DEFINE FILENAME tag_hasType_tagclass_file;
  DEFINE FILENAME organisation_isLocatedIn_place_file;
  DEFINE FILENAME comment_isLocatedIn_place_file;
  DEFINE FILENAME post_isLocatedIn_place_file;
  DEFINE FILENAME person_isLocatedIn_place_file;
  DEFINE FILENAME place_isPartOf_place_file;
  DEFINE FILENAME tagclass_isSubclassOf_tagclass_file;
  DEFINE FILENAME person_knows_person_file;
  DEFINE FILENAME person_likes_comment_file;
  DEFINE FILENAME person_likes_post_file;
  DEFINE FILENAME comment_replyOf_comment_file;
  DEFINE FILENAME comment_replyOf_post_file;
  DEFINE FILENAME person_studyAt_organisation_file;
  DEFINE FILENAME person_workAt_organisation_file;

  // load vertex
  LOAD v_comment_file 
    TO VERTEX Comment VALUES ($0, $0, $1, $2, $3, $4, $5) USING header="true", separator="|";
  LOAD v_post_file
    TO VERTEX Post VALUES ($0, $0, $1, $2, $3, $4, $5, $6, $7) USING header="true", separator="|";
  LOAD v_organisation_file
    TO VERTEX Company VALUES ($0, $0, $2, $3) WHERE $1=="company",
    TO VERTEX University VALUES ($0, $0, $2, $3) WHERE $1=="university" USING header="true", separator="|";
  LOAD v_place_file
    TO VERTEX City VALUES ($0, $0, $1, $2) WHERE $3=="city",
    TO VERTEX Country VALUES ($0, $0, $1, $2) WHERE $3=="country",
    TO VERTEX Continent VALUES ($0, $0, $1, $2) WHERE $3=="continent" USING header="true", separator="|";
  LOAD v_forum_file
    TO VERTEX Forum VALUES ($0, $0, $1, $2) USING header="true", separator="|";
  LOAD v_person_file
    TO VERTEX Person VALUES ($0, $0, $1, $2, $3, $4, $5, $6, $7, SPLIT($8,";"), SPLIT($9,";")) USING header="true", separator="|";
  LOAD v_tag_file
    TO VERTEX Tag VALUES ($0, $0, $1, $2) USING header="true", separator="|";
  LOAD v_tagclass_file
    TO VERTEX TagClass VALUES ($0, $0, $1, $2) USING header="true", separator="|";

  // load edge
  LOAD forum_containerOf_post_file
    TO EDGE Forum_CONTAINER_OF_Post VALUES ($0, $1) USING header="true", separator="|";
  LOAD comment_hasCreator_person_file
    TO EDGE Comment_HAS_CREATOR_Person VALUES ($0, $1) USING header="true", separator="|";
  LOAD post_hasCreator_person_file
    TO EDGE Post_HAS_CREATOR_Person VALUES ($0, $1) USING header="true", separator="|";
  LOAD person_hasInterest_tag_file
    TO EDGE Person_HAS_INTEREST_Tag VALUES ($0, $1) USING header="true", separator="|";
  LOAD forum_hasMember_person_file
    TO EDGE Forum_HAS_MEMBER_Person VALUES ($0, $1, $2) USING header="true", separator="|";
  LOAD forum_hasModerator_person_file
    TO EDGE Forum_HAS_MODERATOR_Person VALUES ($0, $1) USING header="true", separator="|";
  LOAD comment_hasTag_tag_file
    TO EDGE Comment_HAS_TAG_Tag VALUES ($0, $1) USING header="true", separator="|";
  LOAD post_hasTag_tag_file
    TO EDGE Post_HAS_TAG_Tag VALUES ($0, $1) USING header="true", separator="|";
  LOAD forum_hasTag_tag_file
    TO EDGE Forum_HAS_TAG_Tag VALUES ($0, $1) USING header="true", separator="|";
  LOAD tag_hasType_tagclass_file
    TO EDGE Tag_HAS_TYPE_TagClass VALUES ($0, $1) USING header="true", separator="|";
  LOAD organisation_isLocatedIn_place_file
    TO EDGE Company_IS_LOCATED_IN_Country VALUES ($0, $1) WHERE to_int($1) < 111, 
    TO EDGE University_IS_LOCATED_IN_City VALUES ($0, $1) WHERE to_int($1) > 110 USING header="true", separator="|";
  LOAD comment_isLocatedIn_place_file
    TO EDGE Comment_IS_LOCATED_IN_Country VALUES ($0, $1) USING header="true", separator="|";
  LOAD post_isLocatedIn_place_file
    TO EDGE Post_IS_LOCATED_IN_Country VALUES ($0, $1) USING header="true", separator="|";
  LOAD person_isLocatedIn_place_file
    TO EDGE Person_IS_LOCATED_IN_City VALUES ($0, $1) USING header="true", separator="|";
  LOAD place_isPartOf_place_file
    TO EDGE Country_IS_PART_OF_Continent VALUES ($0, $1) WHERE to_int($0) < 111,
    TO EDGE City_IS_PART_OF_Country VALUES ($0, $1) WHERE to_int($0) > 110 USING header="true", separator="|";
  LOAD tagclass_isSubclassOf_tagclass_file
    TO EDGE TagClass_IS_SUBCLASS_OF_TagClass VALUES ($0, $1) USING header="true", separator="|";
  LOAD person_knows_person_file
    TO EDGE Person_KNOWS_Person VALUES ($0, $1, $2) USING header="true", separator="|";
  LOAD person_likes_comment_file
    TO EDGE Person_LIKES_Comment VALUES ($0, $1, $2) USING header="true", separator="|";
  LOAD person_likes_post_file
    TO EDGE Person_LIKES_Post VALUES ($0, $1, $2) USING header="true", separator="|";
  LOAD comment_replyOf_comment_file
    TO EDGE Comment_REPLY_OF_Comment VALUES ($0, $1) USING header="true", separator="|";
  LOAD comment_replyOf_post_file
    TO EDGE Comment_REPLY_OF_Post VALUES ($0, $1) USING header="true", separator="|";
  LOAD person_studyAt_organisation_file
    TO EDGE Person_STUDY_AT_University VALUES ($0, $1, $2) USING header="true", separator="|";
  LOAD person_workAt_organisation_file
    TO EDGE Person_WORK_AT_Company VALUES ($0, $1, $2) USING header="true", separator="|";
}

Overwriting create_load_job.gsql


In [8]:
!gsql create_load_job.gsql

Using graph 'ldbc_snb'
The job load_ldbc_snb is created.


In [9]:
%%writefile run_load_job.sh
#!/bin/bash

###############################################
# Copyright (c)  2015-now, TigerGraph Inc.
# All rights reserved
# It is provided as it is for training purpose.
# Author: mingxi.wu@tigergraph.com
################################################

## change the path below to your ldbc_snb_data path, and remove the '#'
export LDBC_SNB_DATA_DIR=ldbc_snb_data/social_network/

gsql -g ldbc_snb "run loading job load_ldbc_snb using 
v_person_file=\"${LDBC_SNB_DATA_DIR}/person_0_0.csv\",
v_post_file=\"${LDBC_SNB_DATA_DIR}/post_0_0.csv\", 
v_tag_file=\"${LDBC_SNB_DATA_DIR}/tag_0_0.csv\", 
v_place_file=\"${LDBC_SNB_DATA_DIR}/place_0_0.csv\",
v_comment_file=\"${LDBC_SNB_DATA_DIR}/comment_0_0.csv\", 
v_forum_file=\"${LDBC_SNB_DATA_DIR}/forum_0_0.csv\", 
v_organisation_file=\"${LDBC_SNB_DATA_DIR}/organisation_0_0.csv\", 
v_tagclass_file=\"${LDBC_SNB_DATA_DIR}/tagclass_0_0.csv\",
person_knows_person_file=\"${LDBC_SNB_DATA_DIR}/person_knows_person_0_0.csv\", 
comment_replyOf_post_file=\"${LDBC_SNB_DATA_DIR}/comment_replyOf_post_0_0.csv\", 
comment_replyOf_comment_file=\"${LDBC_SNB_DATA_DIR}/comment_replyOf_comment_0_0.csv\", 
post_hasCreator_person_file=\"${LDBC_SNB_DATA_DIR}/post_hasCreator_person_0_0.csv\", 
post_hasTag_tag_file=\"${LDBC_SNB_DATA_DIR}/post_hasTag_tag_0_0.csv\", 
comment_hasCreator_person_file=\"${LDBC_SNB_DATA_DIR}/comment_hasCreator_person_0_0.csv\", 
post_isLocatedIn_place_file=\"${LDBC_SNB_DATA_DIR}/post_isLocatedIn_place_0_0.csv\", 
comment_hasTag_tag_file=\"${LDBC_SNB_DATA_DIR}/comment_hasTag_tag_0_0.csv\", 
comment_isLocatedIn_place_file=\"${LDBC_SNB_DATA_DIR}/comment_isLocatedIn_place_0_0.csv\", 
forum_containerOf_post_file=\"${LDBC_SNB_DATA_DIR}/forum_containerOf_post_0_0.csv\", 
forum_hasMember_person_file=\"${LDBC_SNB_DATA_DIR}/forum_hasMember_person_0_0.csv\", 
forum_hasModerator_person_file=\"${LDBC_SNB_DATA_DIR}/forum_hasModerator_person_0_0.csv\", 
forum_hasTag_tag_file=\"${LDBC_SNB_DATA_DIR}/forum_hasTag_tag_0_0.csv\", 
organisation_isLocatedIn_place_file=\"${LDBC_SNB_DATA_DIR}/organisation_isLocatedIn_place_0_0.csv\",
person_hasInterest_tag_file=\"${LDBC_SNB_DATA_DIR}/person_hasInterest_tag_0_0.csv\", 
person_isLocatedIn_place_file=\"${LDBC_SNB_DATA_DIR}/person_isLocatedIn_place_0_0.csv\", 
person_likes_comment_file=\"${LDBC_SNB_DATA_DIR}/person_likes_comment_0_0.csv\", 
person_likes_post_file=\"${LDBC_SNB_DATA_DIR}/person_likes_post_0_0.csv\", 
person_studyAt_organisation_file=\"${LDBC_SNB_DATA_DIR}/person_studyAt_organisation_0_0.csv\", 
person_workAt_organisation_file=\"${LDBC_SNB_DATA_DIR}/person_workAt_organisation_0_0.csv\", 
place_isPartOf_place_file=\"${LDBC_SNB_DATA_DIR}/place_isPartOf_place_0_0.csv\",
tag_hasType_tagclass_file=\"${LDBC_SNB_DATA_DIR}/tag_hasType_tagclass_0_0.csv\", 
tagclass_isSubclassOf_tagclass_file=\"${LDBC_SNB_DATA_DIR}/tagclass_isSubclassOf_tagclass_0_0.csv\"" 

Overwriting run_load_job.sh


In [10]:
!/bin/bash run_load_job.sh

[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
  JobName: load_ldbc_snb, jobid: ldbc_snb.load_ldbc_snb.file.m1.1592231042478
  Loading log: '/home/tigergraph/tigergraph/logs/restpp/restpp_loader_logs/ldbc_snb/ldbc_snb.load_ldbc_snb.file.m1.1592231042478.log'

Job "ldbc_snb.load_ldbc_snb.file.m1.1592231042478" loading status
[RUNNING] m1 ( Finished: 0 / Total: 31 )
Job "ldbc_snb.load_ldbc_snb.file.m1.1592231042478" loading status
[RUNNING] m1 ( Finished: 0 / Total: 31 )
  [LOADING] /home/tigergraph/snb/ldbc_snb_data/social_network/comment_0_0.csv
  [===========                                                      ] 18%, 83 kl/s
Job "ldbc_snb.load_ldbc_snb.file.m1.1592231042478" loading status
[RUNNING] m1 ( Finished: 0 / Total: 31 )
  [LOADING] /home/tigergraph/snb/ldbc_snb_data/social_network/comm

In [11]:
!gadmin status graph -v

Welcome to TigerGraph Developer Edition, free for non-production, research, or educational use.
verbose is ON
=== graph ===
[m1     ][GRAPH][MSG ] Graph was loaded (/home/tigergraph/tigergraph/gstore/0/part/): partition size is 323.00MiB, IDS size is 104.86MiB, SchemaVersion: 0, VertexCount: 3177176, NumOfSkippedVertices: 0, NumOfDeletedVertices: 0, EdgeCount: 26427510
[m1     ][GRAPH][INIT] True
[INFO   ][GRAPH][MSG ] Above vertex and edge counts are for internal use which show approximate topology size of the local graph partition. Use DML to get the correct graph topology information
[SUMMARY][GRAPH] graph is ready


### Define a function to process queries

In [148]:
import subprocess, json

def gsql(text, graph='ldbc_snb'):
    
    comp = subprocess.run(['gsql', '-g ' + graph, text], text=True, capture_output=True)
    
    try:
        json_object = json.loads(comp.stdout)
    except:
        return comp.stdout.replace('\\n','\n')
    else:
        return json_object

### Verify correct loading

In [13]:
import re

show_vertex = !gsql -g ldbc_snb 'show vertex *'
re.findall('VERTEX (\w*)', ''.join(show_vertex))

['Comment',
 'Post',
 'Company',
 'University',
 'City',
 'Country',
 'Continent',
 'Forum',
 'Person',
 'Tag',
 'TagClass']

In [137]:
import pandas as pd

df = pd.read_csv('ldbc_snb_data/social_network/comment_0_0.csv', sep='|', nrows=5)
df

,id,creationDate,locationIP,browserUsed,content,length
0,1236950581249,2011-08-17T14:26:59.961+0000,92.39.58.88,Chrome,yes,3
1,1236950581250,2011-08-17T11:10:21.570+0000,213.55.127.9,Internet Explorer,thanks,6
2,2061584302085,2012-07-20T05:22:51.283+0000,213.55.127.9,Internet Explorer,LOL,3
3,2061584302086,2012-07-20T16:55:45.373+0000,213.55.127.9,Internet Explorer,I see,5
4,2061584302087,2012-07-20T04:34:17.500+0000,213.55.127.9,Internet Explorer,fine,4


In [138]:
gsql('select * from Comment where primary_id=={}'.format(df["id"][0]))

[{'v_id': '1236950581249',
  'attributes': {'browserUsed': 'Chrome',
   'length': 3,
   'locationIP': '92.39.58.88',
   'id': 1236950581249,
   'creationDate': '2011-08-17 14:26:59',
   'content': 'yes'},
  'v_type': 'Comment'}]

In [139]:
df = pd.read_csv('ldbc_snb_data/social_network/post_0_0.csv', sep='|', nrows=5)
df

,id,imageFile,creationDate,locationIP,browserUsed,language,content,length
0,1236950581248,NaN,2011-08-17T06:05:40.595+0000,49.246.218.237,Firefox,uz,"About Rupert Murdoch, t newer electronic publi...",140
1,3,NaN,2010-02-14T20:30:21.451+0000,119.235.7.103,Firefox,uz,"About Rupert Murdoch, alised US citizen iAbout...",108
2,2061584302084,NaN,2012-07-20T03:48:53.147+0000,119.235.7.103,Firefox,uz,"About Rupert Murdoch, 50s and '60s, he acquAbo...",123
3,1649267441688,NaN,2012-02-27T03:45:05.655+0000,119.235.7.103,Firefox,uz,"About Rupert Murdoch, w York in 1974 to expand...",200
4,1786706395168,NaN,2012-03-21T16:21:11.935+0000,119.235.7.103,Firefox,uz,"About Rupert Murdoch, to the United Kingdom in...",120


In [140]:
gsql('select * from Post where primary_id=={}'.format(df["id"][0]))

[{'v_id': '1236950581248',
  'attributes': {'imageFile': '',
   'browserUsed': 'Firefox',
   'length': 140,
   'locationIP': '49.246.218.237',
   'id': 1236950581248,
   'creationDate': '2011-08-17 06:05:40',
   'lang': 'uz',
   'content': 'About Rupert Murdoch, t newer electronic publishing technoAbout George Frideric Handel,  concertos. Handel was born in 1685,About Kurt Vonne'},
  'v_type': 'Post'}]

In [141]:
df = pd.read_csv('ldbc_snb_data/social_network/organisation_0_0.csv', sep='|')
df.type.value_counts()

university    6380
company       1575
Name: type, dtype: int64

In [142]:
df.loc[df.type == 'company'].head()

,id,type,name,url
0,0,company,Kam_Air,http://dbpedia.org/resource/Kam_Air
1,1,company,Balkh_Airlines,http://dbpedia.org/resource/Balkh_Airlines
2,2,company,Khyber_Afghan_Airlines,http://dbpedia.org/resource/Khyber_Afghan_Airl...
3,3,company,MarcoPolo_Airways,http://dbpedia.org/resource/MarcoPolo_Airways
4,4,company,Pamir_Airways,http://dbpedia.org/resource/Pamir_Airways


In [143]:
gsql('select * from Company where primary_id=={}'.format(
    df.loc[df.type == 'company'].id.iloc[0]))

[{'v_id': '0',
  'attributes': {'name': 'Kam_Air',
   'id': 0,
   'url': 'http://dbpedia.org/resource/Kam_Air'},
  'v_type': 'Company'}]

In [144]:
df.loc[df.type == 'university'].head()

,id,type,name,url
1575,1575,university,Paktia_University,http://dbpedia.org/resource/Paktia_University
1576,1576,university,Baghlan_University,http://dbpedia.org/resource/Baghlan_University
1577,1577,university,Syed_Jamaluddin_Afghan_University,http://dbpedia.org/resource/Syed_Jamaluddin_Af...
1578,1578,university,Bost_University,http://dbpedia.org/resource/Bost_University
1579,1579,university,Aga_Khan_University,http://dbpedia.org/resource/Aga_Khan_University


In [145]:
gsql('select * from University where primary_id=={}'.format(
    df.loc[df.type == 'university'].id.iloc[0]))

[{'v_id': '1575',
  'attributes': {'name': 'Paktia_University',
   'id': 1575,
   'url': 'http://dbpedia.org/resource/Paktia_University'},
  'v_type': 'University'}]

In [146]:
df = pd.read_csv('ldbc_snb_data/social_network/person_0_0.csv', sep='|', nrows=5)
df

,id,firstName,lastName,gender,birthday,creationDate,locationIP,browserUsed,language,email
0,933,Mahinda,Perera,male,1989-12-03,2010-02-14T15:32:10.447+0000,119.235.7.103,Firefox,si;en,Mahinda933@boarderzone.com;Mahinda933@hotmail....
1,1129,Carmen,Lepland,female,1984-02-18,2010-01-28T06:39:58.781+0000,195.20.151.175,Internet Explorer,et;en,Carmen1129@gmail.com;Carmen1129@yahoo.com
2,4194,Hồ Chí,Do,male,1988-10-14,2010-02-15T00:46:17.657+0000,103.2.223.188,Internet Explorer,vi;en,Ho.Chi4194@gmx.com
3,8333,Rahul,Kumar,female,1980-02-02,2010-02-12T12:21:43.365+0000,27.97.186.123,Internet Explorer,mr;hi;en,Rahul8333@gmx.com;Rahul8333@yahoo.com
4,8698,Rahul,Reddy,female,1982-05-29,2010-01-21T10:44:41.479+0000,27.97.237.23,Firefox,as;te;en,Rahul8698@gmail.com;Rahul8698@gmx.com;Rahul869...


In [147]:
gsql('select * from Person where primary_id=={}'.format(df.id.iloc[0]))[0]

{'v_id': '933',
 'attributes': {'birthday': '1989-12-03 00:00:00',
  'firstName': 'Mahinda',
  'lastName': 'Perera',
  'gender': 'male',
  'speaks': ['si', 'en'],
  'browserUsed': 'Firefox',
  'locationIP': '119.235.7.103',
  'id': 933,
  'creationDate': '2010-02-14 15:32:10',
  'email': ['Mahinda933@boarderzone.com',
   'Mahinda933@hotmail.com',
   'Mahinda933@yahoo.com',
   'Mahinda933@zoho.com']},
 'v_type': 'Person'}

### Let's explore

![](https://gblobscdn.gitbook.com/assets%2F-LHvjxIN4__6bA0T-QmU%2F-Lexs1CbDfGjI9T9wQ3X%2F-LexxRgZ1XapakZA1gdh%2FScreen%20Shot%202019-05-15%20at%205.05.00%20PM.png?alt=media&token=f9886e4c-4939-4549-9bf1-f8a24ce59ad2)

In [151]:
print(gsql('select t from Person:s - (isLocatedIn:e) -> City:t where s.primary_id=={}'.format(df.id.iloc[0])))

Encountered " ":" ": "" at line 1, column 21.
Was expecting one of:
    <EOF> 
    "create" ...
    "drop" ...
    "clear" ...
    "init" ...
    "alter" ...
    "gen-data" ...
    "install" ...
    "where" ...
    "@" ...
    "typedef" ...
    "begin" ...
    "change" ...
    "delete" ...
    "decrypt" ...
    "offline2online" ...
    "interpret" ...
    "end" ...
    "resume" ...
    "abort" ...
    "exit" ...
    "export" ...
    "grant" ...
    "group" ...
    "help" ...
    "version" ...
    "import" ...
    "upsert" ...
    "limit" ...
    "ls" ...
    "order" ...
    "partition" ...
    "quit" ...
    "refresh" ...
    "revoke" ...
    "run" ...
    "select" ...
    "set" ...
    "show" ...
    "user" ...
    "use" ...
    "recompile" ...
    "-(" ...
    

